# 0.0. Imports

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt

## 0.1. Helper Functions

In [2]:
def change_date(data, column):
    data[column] = pd.to_datetime(data[column], format='%Y-%m-%d')
    return data[column]

def descrever(df):
    num_attributes = df.select_dtypes(['int64', 'float64'])

    mean = pd.DataFrame(num_attributes.apply(np.mean)).T
    median = pd.DataFrame(num_attributes.apply(np.median)).T

    min_ = pd.DataFrame(num_attributes.apply(np.min)).T
    max_ = pd.DataFrame(num_attributes.apply(np.max)).T
    std = pd.DataFrame(num_attributes.apply(np.std)).T
    range_ =  pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
    skew = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
    kurtosis  = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T
    unique  = pd.DataFrame(num_attributes.apply(lambda x: x.nunique())).T


    metricas = pd.concat([ min_, max_, range_, mean, median, std, skew, kurtosis, unique]).T
    metricas.columns = [ 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis', 'unique']
    return metricas

pd.set_option('display.max_rows', 500)

## 0.2. Load original datasets

In [3]:
df_geral = pd.read_csv('../data/portfolio_geral.csv', encoding='UTF-8')
df_cliente = pd.read_csv('../data/portfolio_clientes.csv', encoding='UTF-8')
df_comunicados = pd.read_csv('../data/portfolio_comunicados.csv', encoding='UTF-8')
df_tpv = pd.read_csv('../data/portfolio_tpv.csv', encoding='UTF-8')


# 1.0. Mudança de tipos

In [4]:
#mudança de tipos

#dates
dates = ['dt_contrato', 'dt_desembolso','dt_vencimento', 'dt_wo', 'dt_ref_portfolio' ]
for date in dates:
    df_geral[date] = change_date(df_geral, date)

df_comunicados['data_acao'] = change_date(df_comunicados, 'data_acao')
df_comunicados['dt_ref_portfolio'] = change_date(df_comunicados, 'dt_ref_portfolio')

df_tpv['dt_transacao'] = change_date(df_tpv, 'dt_transacao')




# 2.0. Tratamentos e Joins

## 2.1. Split portfolio geral

In [5]:
#split geral -> geral e contrato
df_contrato = df_geral[['contrato_id', 'nr_documento', 'safra', 'dt_contrato', 'dt_desembolso',
                        'dt_vencimento','dt_wo', 'prazo', 'vlr_desembolsado',
                        'vlr_tarifa', 'juros_mes','juros_diario']].drop_duplicates('contrato_id')

df_geral = df_geral.drop(['safra', 'dt_contrato', 'dt_desembolso',
                        'dt_vencimento','dt_wo', 'prazo', 'vlr_desembolsado',
                        'vlr_tarifa', 'juros_mes','juros_diario'], axis=1)

## 2.2. Filtrar linhas do df_geral

In [6]:
#filtrar geral -> manter somente primeira linha de settled

#sort df_geral por id e date
df_geral = df_geral.sort_values(['contrato_id', 'dt_ref_portfolio']).reset_index(drop=True)



#pega todas linhas com status quitado
aux = df_geral[df_geral['status_contrato']=='Settled']

#pega data minima de cada contrato quitado
aux0 = aux[['contrato_id', 'dt_ref_portfolio']].groupby('contrato_id').min().reset_index()

#juntar outras features 
aux0 = pd.merge(aux0, df_geral, on=['contrato_id', 'dt_ref_portfolio'], how='inner')

#remover todos status quitados de df_geral
df_geral = df_geral[df_geral['status_contrato']!='Settled']

#colocar de volta apenas o primeiro registro do contrato quitado
df_geral = pd.concat([df_geral, aux0])



In [7]:
#encontrar dados estranhos
descrever(df_geral)

,min,max,range,mean,median,std,skew,kurtosis,unique
perc_retencao,0.0114,1.20,1.1886,0.207289,0.156,0.242711,3.425979,11.127067,2301.0
vlr_pgto_realizado,-3195.5600,210216.25,213411.8100,86.328751,0.000,452.963586,127.412612,35987.554675,111177.0
vlr_pgto_esperado,0.5700,1134.03,1133.4600,112.022508,70.700,124.339717,2.586350,8.586397,12604.0
vlr_saldo_devedor,-44613.3700,750102.75,794716.1200,29216.660267,15216.420,41036.658142,3.663851,21.157096,2878673.0
vlr_saldo_devedor_esperado,0.0000,401338.69,401338.6900,19754.802032,9216.805,30367.406702,3.450363,16.585532,2948472.0
dsp,0.0000,791.00,791.0000,21.725396,1.000,62.933864,4.474875,24.865537,792.0
dspp,0.0000,833.00,833.0000,58.527839,3.000,114.092322,2.513723,6.531199,834.0
flag_transacao,0.0000,1.00,1.0000,0.568374,1.000,0.495303,-0.276088,-1.923776,2.0


In [8]:
#limpar dados estranhos

# vlr_pgto_realizado<0
linhas = len(df_geral[df_geral['vlr_pgto_realizado']<0])
print(f'Linhas removidas para valores de pagamento negativo: {linhas}')
df_geral = df_geral[df_geral['vlr_pgto_realizado']>=0]

# vlr_saldo_devedor <0
linhas = len(df_geral[df_geral['vlr_saldo_devedor']<0])
print(f'Linhas removidas para valores de pagamento negativo: {linhas}')
df_geral = df_geral[df_geral['vlr_saldo_devedor']>=0]


# df_geral[df_geral['vlr_saldo_devedor']<0]

Linhas removidas para valores de pagamento negativo: 23
Linhas removidas para valores de pagamento negativo: 1591


## 2.3. Merge -  portifólio geral e de comunicados

In [9]:
#merge geral e comunicados
df_geral_comunicado = pd.merge(df_geral, df_comunicados, on=['contrato_id', 'dt_ref_portfolio'], how='left')


## 2.4. Merge - portfólio geral e de tpv

In [10]:
#encontrar dados estranhos tpv
descrever(df_tpv)

,min,max,range,mean,median,std,skew,kurtosis,unique
qtd_transacoes,-2.0,1245.00,1247.00,15.906579,7.0,26.153359,6.003932,70.753434,705.0
vlr_tpv,-125000.0,176880.93,301880.93,887.497885,430.0,1664.209969,10.497490,362.253612,492049.0


In [11]:
#limpar dados estranhos
linhas = len(df_tpv[df_tpv['qtd_transacoes']<0])
print(f'Linhas removidas para valores de quantidade de transação negativo: {linhas}')
df_tpv = df_tpv[df_tpv['qtd_transacoes']>=0]

linhas = len(df_tpv[df_tpv['vlr_tpv']<0])
print(f'Linhas removidas para valores de transação de transação negativo: {linhas}')
df_tpv = df_tpv[df_tpv['vlr_tpv']>=0]


Linhas removidas para valores de quantidade de transação negativo: 7
Linhas removidas para valores de transação de transação negativo: 1238


In [12]:
#merge geral e tpv
df_geral = pd.merge(df_geral, df_tpv.rename(columns = {'dt_transacao':'dt_ref_portfolio'}), on=['nr_documento', 'dt_ref_portfolio'], how='left')

## 2.5. Merge - df_contrato e portfólio de clientes


In [13]:
#merge contratos e clientes
df_contrato = pd.merge(df_contrato, df_cliente, on='nr_documento', how='inner')

# 3.0. Feature Engineering

### 3.1. Features em df_geral

In [14]:
#valor debitado desconsiderando pagamento 
df_geral['debito_sem_pagamento'] = df_geral['vlr_saldo_devedor'] + df_geral['vlr_pgto_realizado']


#valor pago tpv = valor * perc_retencao/100
df_geral['valor_pago_tpv'] = df_geral['vlr_tpv'] * df_geral['perc_retencao']/100


### 3.2. Features em df_contrato

In [15]:
#extrair ultima data de cada contrato
last_date = df_geral[['contrato_id', 'dt_ref_portfolio']].groupby('contrato_id').max().reset_index()
last_date = pd.merge(last_date, df_geral[['contrato_id', 'dt_ref_portfolio','status_contrato']], on=['contrato_id', 'dt_ref_portfolio'], how='inner')

# #conseguir o ultimo status de contrato
df_contrato = pd.merge(df_contrato, last_date.drop('dt_ref_portfolio', axis=1), on='contrato_id', how='inner')

#mean dsp / contrato
aux = df_geral[['dsp', 'contrato_id']].groupby('contrato_id').mean().reset_index()
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='left')

#mean dspp / contrato
aux = df_geral[['dspp', 'contrato_id']].groupby('contrato_id').mean().reset_index()
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='left')

#sum vlr pago realizado / contrato
aux = df_geral[['vlr_pgto_realizado', 'contrato_id']].groupby('contrato_id').sum().reset_index()
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='left')

#sum vlr pago tpv realizado / contrato
aux = df_geral[['valor_pago_tpv', 'contrato_id']].groupby('contrato_id').sum().reset_index()
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='left')

#valor total pago / contrato
df_contrato['valor_total_pago'] = df_contrato['vlr_pgto_realizado'] + df_contrato['valor_pago_tpv']

#valor final que deveria ser pago desconsiderando pagamento
aux = df_geral[['debito_sem_pagamento','contrato_id']].groupby('contrato_id').max().reset_index()
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='left')

#dias esperado de contrato
df_contrato['tempo_esperado'] = (df_contrato['dt_vencimento'] - df_contrato['dt_desembolso']).dt.days

# # #dias efetivos de contrato 
aux = df_geral[['contrato_id', 'dt_ref_portfolio']].groupby('contrato_id').count().rename(columns={'dt_ref_portfolio': 'dias_de_contrato'}).reset_index()
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='left')

#acoes totais
aux = df_geral_comunicado[['contrato_id', 'status']].groupby('contrato_id').count().reset_index().rename(columns={'status': 'n_acoes_total'})
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='inner')

# #acoes entregues
aux = df_geral_comunicado [df_geral_comunicado['status']!='NAO ENTREGUE'] [['contrato_id', 'status']].groupby('contrato_id').count().reset_index().rename(columns={'status': 'n_acoes_entregues'})
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='inner')

# 4.0. Criação de tabelas

## 4.1. Criação da df_mensagem

Um dataframe chamado df_mensagem foi criado apenas com as linhas do df_geral que continha uma comunicação com status diferente de “NAO ENTREGUE” e os 5 registros seguintes. O objetivo desse dataframe é observar o efeito da comunicação nos 5 dias seguintes, através do dsp e dspp. Dessa vez o enfoque é descobrir a curva ideal de vezes que se deve acionar o cliente.


In [16]:
# #pegar linha da ação
df_sorted = df_geral_comunicado.sort_values(['contrato_id', 'dt_ref_portfolio']).reset_index(drop=True)

#get index of not null messages
index_mensagem = df_sorted.loc[~df_sorted['status'].isna(), :].index

index_mensagem_expanded = index_mensagem
#create index list expanded

index_aux = index_mensagem + 1
index_mensagem_expanded = index_mensagem_expanded.append(index_aux)
index_aux+=1
index_mensagem_expanded = index_mensagem_expanded.append(index_aux)
index_aux+=1
index_mensagem_expanded = index_mensagem_expanded.append(index_aux)
index_aux+=1
index_mensagem_expanded = index_mensagem_expanded.append(index_aux)
index_aux+=1
index_mensagem_expanded = index_mensagem_expanded.append(index_aux)


#get df mensagem
df_mensagem = df_sorted.iloc[index_mensagem_expanded, :].sort_values(['contrato_id', 'dt_ref_portfolio']).drop_duplicates().reset_index(drop=True)

#get index das mensagens 
index_mensagem = df_mensagem.loc[df_mensagem['status'].isin(['LIDO', 'RESPONDIDO']), :].index

#create eficiencia
for index, i in enumerate(index_mensagem):
#     print(f'{index}/{len(index_mensagem)}')
    
    if df_mensagem.loc[i,'status']: #se status existe

        if df_mensagem.loc[i,'dsp']==0:   #se dsp ==0, passa
            pass

        else:                               #se dsp!=0, verifica os proximos 5 se vai ser igual a 0
            for i2 in range(i+1, i+6):
                if df_mensagem.loc[i2, 'dsp']==0:

                    df_mensagem.loc[i,'eficiencia']=1
    
                    if df_mensagem.loc[i2, 'dspp']==0:
                        df_mensagem.loc[i,'eficiencia']=2
                        
                else: 
                    pass
        

#fill null with 0
df_mensagem['eficiencia'] = df_mensagem['eficiencia'].fillna(0)



## 4.2 Criação da df_lido


Um dataframe chamado df_lido foi criado apenas com as linhas do df_geral que continha uma comunicação com status “LIDO”/”RESPONDIDO” e os 5 registros seguintes. O objetivo desse dataframe é observar o efeito da comunicação nos 5 dias seguintes, através do dsp e dspp.

In [23]:
# #pegar linha da ação
df_sorted = df_geral_comunicado.sort_values(['contrato_id', 'dt_ref_portfolio']).reset_index(drop=True)

#get index of not null messages
index_lido = df_sorted.loc[df_sorted['status'].isin(['LIDO','RESPONDIDO']), :].index

index_lido_expanded = index_lido
#create index list expanded

index_aux = index_lido + 1
index_lido_expanded = index_lido_expanded.append(index_aux)
index_aux+=1
index_lido_expanded = index_lido_expanded.append(index_aux)
index_aux+=1
index_lido_expanded = index_lido_expanded.append(index_aux)
index_aux+=1
index_lido_expanded = index_lido_expanded.append(index_aux)
index_aux+=1
index_lido_expanded = index_lido_expanded.append(index_aux)


#get df lido
df_lido = df_sorted.iloc[index_lido_expanded, :].sort_values(['contrato_id', 'dt_ref_portfolio']).drop_duplicates().reset_index(drop=True)

#get index das mensagens 
index_lido = df_lido.loc[df_lido['status'].isin(['LIDO', 'RESPONDIDO']), :].index

#create eficiencia
for index, i in enumerate(index_lido):
#     print(f'{index}/{len(index_lido)}')
    
    if df_lido.loc[i,'status']: #se status existe

        if df_lido.loc[i,'dsp']==0:   #se dsp ==0, passa
            pass

        else:                               #se dsp!=0, verifica os proximos 5 se vai ser igual a 0
            for i2 in range(i+1, i+6):
                if df_lido.loc[i2, 'dsp']==0:

                    df_lido.loc[i,'eficiencia']=1
    
                    if df_lido.loc[i2, 'dspp']==0:
                        df_lido.loc[i,'eficiencia']=2
                        
                else: 
                    pass
        

#fill null with 0
df_lido['eficiencia'] = df_lido['eficiencia'].fillna(0)



In [24]:
df_lido.head(100)

,contrato_id,dt_ref_portfolio,nr_documento,status_contrato,perc_retencao,vlr_pgto_realizado,vlr_pgto_esperado,vlr_saldo_devedor,vlr_saldo_devedor_esperado,dsp,dspp,flag_transacao,data_acao,tipo_acao,acao,status,eficiencia
0,000180509391a5ac66ff83cae603ffb8,2020-12-29,7996daab1bbe000bb5d1cc1bf317f390,Active,0.060,0.00,27.45,1960.10,4929.02,5,5,0,2020-12-29,EMAIL,campanhaobservacao,LIDO,0.0
1,000180509391a5ac66ff83cae603ffb8,2020-12-29,7996daab1bbe000bb5d1cc1bf317f390,Active,0.060,0.00,27.45,1960.10,4929.02,5,5,0,2020-12-29,HSM,campanhaobservacao,NAO ENTREGUE,0.0
2,000180509391a5ac66ff83cae603ffb8,2020-12-30,7996daab1bbe000bb5d1cc1bf317f390,Active,0.060,0.00,27.45,1965.75,4915.73,6,6,0,NaT,NaN,NaN,NaN,0.0
3,000180509391a5ac66ff83cae603ffb8,2020-12-31,7996daab1bbe000bb5d1cc1bf317f390,Active,0.060,0.00,27.45,1971.39,4902.41,7,7,0,NaT,NaN,NaN,NaN,0.0
4,000180509391a5ac66ff83cae603ffb8,2021-01-01,7996daab1bbe000bb5d1cc1bf317f390,Active,0.060,0.00,27.45,1977.07,4889.06,8,8,0,NaT,NaN,NaN,NaN,0.0
5,000180509391a5ac66ff83cae603ffb8,2021-01-02,7996daab1bbe000bb5d1cc1bf317f390,Active,0.060,0.00,27.45,1982.75,4875.66,9,9,0,NaT,NaN,NaN,NaN,0.0
6,000c35a61297edadc2842f6d5b4028e1,2021-02-13,1191ebfa94d3ca2e8a02f696aafde4a4,Active,0.180,0.00,106.19,25979.28,26170.22,5,5,1,2021-02-15,HSM,campanhaobservacao,LIDO,1.0
7,000c35a61297edadc2842f6d5b4028e1,2021-02-14,1191ebfa94d3ca2e8a02f696aafde4a4,Active,0.180,0.00,106.19,26021.48,26106.53,6,6,1,NaT,NaN,NaN,NaN,0.0
8,000c35a61297edadc2842f6d5b4028e1,2021-02-15,1191ebfa94d3ca2e8a02f696aafde4a4,Active,0.180,0.00,106.19,26063.75,26042.73,7,7,1,NaT,NaN,NaN,NaN,0.0
9,000c35a61297edadc2842f6d5b4028e1,2021-02-16,1191ebfa94d3ca2e8a02f696aafde4a4,Active,0.180,0.00,106.19,26106.09,25978.83,8,8,1,NaT,NaN,NaN,NaN,0.0


## 4.3. Criação do df_inadimplentes

In [ ]:
df_inadimplentes = df_contrato[df_contrato['status_contrato']!='Settled']

# 5.0. Exportar para csv

In [25]:
# df_contrato.to_csv('../data/new_dfs/df_contrato.csv')
# df_geral.to_csv('../data/new_dfs/df_geral.csv')
# df_geral_comunicado.to_csv('../data/new_dfs/df_geral_comunicado.csv')
# df_lido.to_csv('../data/new_dfs/df_lido.csv')
# df_mensagem.to_csv('../data/new_dfs/df_mensagem.csv')
# df_inadimplentes.to_csv('../data/new_dfs/df_inadimplentes.csv')